In [1]:
import pandas as pd
import numpy as np
import copy
import random
from math import sqrt
import scipy.stats as sts

from tqdm import tqdm_notebook

from supplement_package import game
from supplement_package import variables_pecan

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [3]:
from supplement_package.game.stackelberg import StackelbergPlayer

In [4]:
import gurobipy as gp

from supplement_package.gurobi_implementation.gurobi import GurobiSolution

## Data preliminaries 

In [5]:
agent_keys = [661, 1642, 2335, 2361, 2818, 3039, 3456, 3538, 4031, 4373, 4767, 5746, 6139, 7536, 7719, 7800, 7901, 7951, 8156, 8386, 8565, 9019, 9160, 9922, 9278]

dataframe_dict = dict()
for key in agent_keys:
    dataframe_dict.update({key : pd.read_csv('/Users/ishilov/Documents/risk_paper/risk_paper/data/df_{}.csv'.format(key))})

community_size = len(dataframe_dict)

for key in agent_keys:
    cond_min = (dataframe_dict[key]['demand'].quantile(0.001) <= dataframe_dict[key]['demand'])
    cond_max = (dataframe_dict[key]['demand'] <= dataframe_dict[key]['demand'].quantile(0.999))
    dataframe_dict[key] = dataframe_dict[key][cond_min & cond_max]

In [6]:
def generate_new_params(sample_size, community_size):
    res = {}

    for sample in range(sample_size):
        A_tilde = [random.uniform(0,1) for i in range(community_size)]
        B_tilde = [random.uniform(0,1) for i in range(community_size)]

        a = [random.uniform(0,1) for i in range(community_size)]
        b = [random.uniform(0,1) for i in range(community_size)]
        d = [random.uniform(0,1) for i in range(community_size)]

        #d_target = [[random.uniform(0,8) for j in range(len(probabilities))] for i in range(community_size)]
        #g_res = [[random.uniform(0,3) for j in range(len(probabilities))] for i in range(community_size)]

        #g_res = np.array(g_res)
        #d_target = np.array(d_target)

        risk_aversion = [random.uniform(0,1) for i in range(community_size)]

        res.update({sample : {'A_tilde' : A_tilde,
                        'B_tilde' : B_tilde,
                        'a' : a,
                        'b' : b,
                        'd' : d,
                        'risk_aversion' : risk_aversion}})

    res_reformed = {(i, key) : res[i][key] for i in range(sample_size) for key in res[0].keys()}
    mindx = pd.MultiIndex.from_tuples(res_reformed.keys())
    df = pd.DataFrame(list(res_reformed.values()), index = mindx)
    df.to_csv(f'../data/param_{sample_size}.csv')


In [7]:
def text_to_adj_matrix(matrix_path):
    res = []
    with open(matrix_path) as file:
        for s in file:
            string = ''.join(s.strip().strip(',').split(', '))
            lst_temp = [int(sym) for sym in string]

            res.append(lst_temp)

    return res

In [8]:
#generate_new_params(2000)

In [9]:
def read_df_param(sample_size):
    df_param = pd.read_csv(f'../data/param_{sample_size}.csv')
    df_param.rename({'Unnamed: 0' : 'Sample', 'Unnamed: 1' : 'Parameter'}, axis=1, inplace= True)
    df_param.set_index(['Sample', 'Parameter'], inplace=True)

    return df_param

In [10]:
def distribution_build(sample_size, agent_keys):
    res = {}
    
    for key in agent_keys:
        #chunks_demand = int(demand_dict[key][0].size / sample_size)
        #chunks_generation = int(solar_dict[key][0].size / sample_size)


        #probas_demand = [np.trapz(demand_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            demand_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_demand)]

        #probas_generation = [np.trapz(generation_dict[key][1][i * sample_size : (i + 1) * sample_size],
        #                            generation_dict[key][0][i * sample_size : (i + 1) * sample_size])
        #                            for i in range(chunks_generation)]

        #res_demand = plt.hist(np.random.choice(demand_dict[key][0], size = sample_size, p = probabilities), bins = int(sample_size / 2))
        #probas_update = res_demand[0] / res_demand[0].sum() if key == 661 else res[661]['probabilities']
        #res.update({key : 
        #            {'values' : res_demand[1], 
        #            'probabilities' : probas_update}})

        hist_demand = plt.hist(dataframe_dict[key]['demand'], bins = sample_size)
        probas_demand, values_demand = hist_demand[0], hist_demand[1]
        probas_demand = probas_demand / probas_demand.sum()

        if 'solar' in dataframe_dict[key].columns:
            hist_solar = plt.hist(dataframe_dict[key][dataframe_dict[key]['solar'] >= 0]['solar'], bins = sample_size)
            probas_solar , values_solar  = hist_solar[0], hist_solar[1]
            probas_solar = probas_demand / probas_demand.sum()

        res.update({key : 
                    {'probas_demand' : probas_demand,
                    'values_demand' : values_demand,
                    'probas_solar' : probas_solar,
                    'values_solar' : values_solar}})

    return res 

In [11]:
def scenario_sampling(sample_size, agent_keys, main_key = 661):
    distribution = distribution_build(sample_size, agent_keys)

    probabilities = distribution[main_key]['probas_demand']

    d_target = []
    g_res = []
    for key in agent_keys:
        d_target.append(distribution[key]['values_demand'][:-1])
        g_res.append(distribution[key]['values_solar'][:-1])

    return probabilities, d_target, g_res

In [12]:
def param_input(df, index = 0):
    A_tilde = list(df.loc[index].loc['A_tilde'])
    B_tilde = list(df.loc[index].loc['B_tilde'])
    a = list(df.loc[index].loc['a'])
    b = list(df.loc[index].loc['b'])
    d = list(df.loc[index].loc['d'])
    risk_aversion = list(df.loc[index].loc['risk_aversion'])

    return A_tilde, B_tilde, a, b, d, risk_aversion

In [13]:
param_amount = 2000

#generate_new_params(param_amount,community_size=community_size)
df_param = read_df_param(param_amount)

In [14]:
def sample_to_csv(scenario_amount, probabilities, d_target, g_res):
    pd.DataFrame(d_target).to_csv(f'../data/df_d_target_{scenario_amount}.csv')
    pd.DataFrame(g_res).to_csv(f'../data/df_g_res_{scenario_amount}.csv')
    pd.DataFrame(probabilities).to_csv(f'../data/df_probabilities_{scenario_amount}.csv')

In [15]:
def sample_from_csv(scenario_amount):
    df_d_target = pd.read_csv(f'../data/df_d_target_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_g_res = pd.read_csv(f'../data/df_g_res_{scenario_amount}.csv').drop('Unnamed: 0', axis = 1)
    df_probabilities =pd.read_csv('../data/df_probabilities_100.csv').drop('Unnamed: 0', axis = 1)
    
    probabilities = df_probabilities.values.squeeze()


    d_target = []
    for _, row in df_d_target.iterrows():
        d_target.append(row.values)

    g_res = []
    for _, row in df_g_res.iterrows():
        g_res.append(row.values)

    return probabilities, d_target, g_res
    

In [16]:
probabilities, d_target, g_res = sample_from_csv(100)
A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param)

In [17]:
def agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res):
    agents = []
    StackelbergPlayer.community_size = community_size
    StackelbergPlayer.probabilities = probabilities

    epsilon = 0.001
    alpha = [[proba/(1 - min(risk_aversion)) for proba in probabilities] for i in range(community_size)]
    #alpha = [[0.2 for proba in probabilities] for i in range(community_size)]
    gamma = [100000 for proba in probabilities]

    j_max = [10 for i in range(community_size)]

    for i in range(community_size):
        agent = StackelbergPlayer(i, d_target[i], g_res[i], a[i], b[i], d[i], 
                    A_tilde[i], B_tilde[i], D_min[i], D_max[i], 
                    G_min[i], G_max[i], risk_aversion[i], Kappa[i], Cost[i], connection_matrix[i],
                    probabilities = probabilities,
                    alpha = alpha[i], 
                    gamma = gamma, 
                    insurance_bound=j_max[i])
        
        agents.append(agent)

    return agents

In [18]:
def vars_to_dict(model, list_vars):
    var_names = []
    
    for var in model.getVars():
        var_names.append(var.VarName)

    dict_res = {}
    for name, var in zip(var_names, list_vars):
        dict_res.update({name : var})

    return dict_res 


In [19]:
def gurobi_experiment(df_param, index, err_track, agents, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity = 0):
    A_tilde, B_tilde, a, b, d, risk_aversion = param_input(df_param, index = index)

    if solution_type == 'centralized_optimistic':
        agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)

    model_1 = gp.Model()
    setup = GurobiSolution(agents=agents,
                    model = model_1,
                    solution_type=solution_type)

    model_1.setParam('OutputFlag', verbosity)

    setup.build_model()

    try:
        model_1.optimize()

        list_vars = model_1.X
        dict_vars = vars_to_dict(model_1, list_vars)
        objective_val = model_1.getObjective().getValue()
        
        
        
    except:
        err_track.append(index)
        list_vars = ['err']
        objective_val = 'err'
        dict_vars = vars_to_dict(model_1, list_vars)

    return dict_vars, objective_val, model_1

In [20]:
D_min = [0 for i in range(community_size)]
D_max = [dataframe_dict[i].demand.max() for i in dataframe_dict.keys()]

G_min = [0 for i in range(community_size)]
G_max = [dataframe_dict[i].grid.max() for i in dataframe_dict.keys()]

Kappa = [[10 if i!=j else 0 for i in range(community_size)] for j in range(community_size)]

Cost = [[1 for i in range(community_size)] for j in range(community_size)]


In [21]:
connection_matrix_1 = text_to_adj_matrix('../matrices/matrix_1.txt')

In [29]:
def experiment(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res,
                df_param, solution_type='centralized_optimistic', verbosity = 0):

    if solution_type == 'centralized_optimistic':
        agents = agents_list_optimistic_total(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix, d_target, g_res)

    total_results= {}
    err_track = []
    for index in tqdm_notebook(df_param.index.levels[0]):
        print(index)
        vars, objective, model = gurobi_experiment(df_param, index, err_track, agents, connection_matrix, probabilities, d_target, g_res, solution_type, verbosity)
        total_results.update({index: {'vars' : vars,
                                    'objective' : objective}})


    results_vars = pd.DataFrame(data = [list(total_results[0]['vars'].values())],
                                        columns=total_results[0]['vars'].keys())

    results_vars['objective'] = total_results[0]['objective']

    for i in tqdm_notebook(range(1, len(total_results))):
        df_temp = pd.DataFrame(data = [list(total_results[i]['vars'].values())],
                                            columns=total_results[i]['vars'].keys())
                                
        df_temp['objective'] = total_results[i]['objective']

        results_vars = pd.concat([results_vars, df_temp], ignore_index=True)

    return total_results, results_vars, model

## Experiment

In [30]:
res_opt, df_opt, model_opt = experiment(A_tilde, B_tilde, a, b, d, risk_aversion, probabilities, connection_matrix_1, d_target, g_res,
            df_param, solution_type='centralized_optimistic')

/var/folders/98/gq_0lybs5k55th2hc4jstmyw00m7tv/T/ipykernel_38063/2189923880.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm_notebook(df_param.index.levels[0]):


  0%|          | 0/2000 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9


KeyboardInterrupt: 

In [31]:
df_opt

,D_0_0,D_0_1,D_0_2,D_0_3,D_0_4,D_0_5,D_0_6,D_0_7,D_0_8,D_0_9,...,u_24_91,u_24_92,u_24_93,u_24_94,u_24_95,u_24_96,u_24_97,u_24_98,u_24_99,objective
0,0.028881,0.084397,0.139827,0.195166,0.250411,0.30556,0.360608,0.415552,0.47041,0.529171,...,8.943112e-08,6.497665e-08,1.001427e-07,6.535665e-08,1.005693e-07,5.810470e-08,9.069730e-08,1.330026e-07,1.331035e-07,1.395003
1,0.028881,0.084397,0.139827,0.195166,0.250411,0.30556,0.360608,0.415552,0.47041,0.529171,...,8.943112e-08,6.497665e-08,1.001427e-07,6.535665e-08,1.005693e-07,5.810470e-08,9.069730e-08,1.330026e-07,1.331035e-07,1.395003
2,0.028881,0.084397,0.139827,0.195166,0.250411,0.30556,0.360608,0.415552,0.47041,0.529171,...,8.943112e-08,6.497665e-08,1.001427e-07,6.535665e-08,1.005693e-07,5.810470e-08,9.069730e-08,1.330026e-07,1.331035e-07,1.395003
3,0.028881,0.084397,0.139827,0.195166,0.250411,0.30556,0.360608,0.415552,0.47041,0.529171,...,8.943112e-08,6.497665e-08,1.001427e-07,6.535665e-08,1.005693e-07,5.810470e-08,9.069730e-08,1.330026e-07,1.331035e-07,1.395003
4,0.028881,0.084397,0.139827,0.195166,0.250411,0.30556,0.360608,0.415552,0.47041,0.529171,...,8.943112e-08,6.497665e-08,1.001427e-07,6.535665e-08,1.005693e-07,5.810470e-08,9.069730e-08,1.330026e-07,1.331035e-07,1.395003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.028881,0.084397,0.139827,0.195166,0.250411,0.30556,0.360608,0.415552,0.47041,0.529171,...,8.943112e-08,6.497665e-08,1.001427e-07,6.535665e-08,1.005693e-07,5.810470e-08,9.069730e-08,1.330026e-07,1.331035e-07,1.395003
1996,0.028881,0.084397,0.139827,0.195166,0.250411,0.30556,0.360608,0.415552,0.47041,0.529171,...,8.943112e-08,6.497665e-08,1.001427e-07,6.535665e-08,1.005693e-07,5.810470e-08,9.069730e-08,1.330026e-07,1.331035e-07,1.395003
1997,0.028881,0.084397,0.139827,0.195166,0.250411,0.30556,0.360608,0.415552,0.47041,0.529171,...,8.943112e-08,6.497665e-08,1.001427e-07,6.535665e-08,1.005693e-07,5.810470e-08,9.069730e-08,1.330026e-07,1.331035e-07,1.395003
1998,0.028881,0.084397,0.139827,0.195166,0.250411,0.30556,0.360608,0.415552,0.47041,0.529171,...,8.943112e-08,6.497665e-08,1.001427e-07,6.535665e-08,1.005693e-07,5.810470e-08,9.069730e-08,1.330026e-07,1.331035e-07,1.395003


In [34]:
g_res

[array([0.     , 0.05545, 0.1109 , 0.16635, 0.2218 , 0.27725, 0.3327 ,
        0.38815, 0.4436 , 0.49905, 0.5545 , 0.60995, 0.6654 , 0.72085,
        0.7763 , 0.83175, 0.8872 , 0.94265, 0.9981 , 1.05355, 1.109  ,
        1.16445, 1.2199 , 1.27535, 1.3308 , 1.38625, 1.4417 , 1.49715,
        1.5526 , 1.60805, 1.6635 , 1.71895, 1.7744 , 1.82985, 1.8853 ,
        1.94075, 1.9962 , 2.05165, 2.1071 , 2.16255, 2.218  , 2.27345,
        2.3289 , 2.38435, 2.4398 , 2.49525, 2.5507 , 2.60615, 2.6616 ,
        2.71705, 2.7725 , 2.82795, 2.8834 , 2.93885, 2.9943 , 3.04975,
        3.1052 , 3.16065, 3.2161 , 3.27155, 3.327  , 3.38245, 3.4379 ,
        3.49335, 3.5488 , 3.60425, 3.6597 , 3.71515, 3.7706 , 3.82605,
        3.8815 , 3.93695, 3.9924 , 4.04785, 4.1033 , 4.15875, 4.2142 ,
        4.26965, 4.3251 , 4.38055, 4.436  , 4.49145, 4.5469 , 4.60235,
        4.6578 , 4.71325, 4.7687 , 4.82415, 4.8796 , 4.93505, 4.9905 ,
        5.04595, 5.1014 , 5.15685, 5.2123 , 5.26775, 5.3232 , 5.37865,
      